# 05. 시각화 및 차트 생성

이 노트북에서 다루는 내용:
- 쿼리 결과로부터 차트 생성
- 다양한 유형의 시각화
- 데이터 기반 자동 시각화
- Plotly를 사용한 인터랙티브 차트

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection
from src.utils.text2sql_utils import execute_text2sql
from src.utils.viz_utils import (
    create_bar_chart,
    create_line_chart,
    create_pie_chart,
    create_scatter_plot,
    auto_visualize
)
import pandas as pd

print("✓ Libraries imported successfully")

In [ ]:
# 데이터베이스 초기화
db = DatabaseConnection()

## 1. 막대 차트 - 지역별 매출

In [ ]:
query = "What is the total sales amount by region?"
result = execute_text2sql(db, query)

if result['success']:
    df = result['results']
    fig = create_bar_chart(df, 'region', 'sum', title='Total Sales by Region')
    fig.show()
else:
    print(f"Error: {result['error']}")

## 2. 파이 차트 - 부서별 직원

In [ ]:
query = "How many employees are in each department?"
result = execute_text2sql(db, query)

if result['success']:
    df = result['results']
    # Assuming the result has department_name and count columns
    if len(df.columns) >= 2:
        fig = create_pie_chart(df, df.columns[0], df.columns[1], 
                              title='Employee Distribution by Department')
        fig.show()
else:
    print(f"Error: {result['error']}")

## 3. 산점도 - 급여 분석

In [ ]:
query = "SELECT e.employee_id, e.salary, e.job_title, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id"
df = db.execute_query_df(query)

fig = create_scatter_plot(df, 'employee_id', 'salary', color_col='department_name',
                         title='Employee Salary Distribution')
fig.show()

## 4. 시계열 - 시간별 매출

In [ ]:
query = "SELECT sale_date, SUM(total_amount) as daily_total FROM sales GROUP BY sale_date ORDER BY sale_date"
df = db.execute_query_df(query)

# sale_date를 datetime으로 변환
df['sale_date'] = pd.to_datetime(df['sale_date'])

fig = create_line_chart(df, 'sale_date', 'daily_total', 
                       title='Daily Sales Trend')
fig.show()

## 5. 자동 시각화

In [ ]:
# 시스템이 자동으로 최적의 시각화 선택하도록 함
queries = [
    "What is the average salary by department?",
    "Show me sales by category",
    "List all projects with their budgets"
]

for query in queries:
    print(f"\nQuery: {query}")
    result = execute_text2sql(db, query)
    
    if result['success']:
        viz = auto_visualize(result['results'], query)
        if viz is not None and hasattr(viz, 'show'):
            viz.show()
        else:
            display(viz)
    else:
        print(f"Error: {result['error']}")

## 6. 쿼리와 시각화 통합 파이프라인

In [ ]:
def query_and_visualize(natural_query: str):
    """Complete pipeline: natural language -> SQL -> results -> chart"""
    print(f"Query: {natural_query}")
    print("="*60)
    
    result = execute_text2sql(db, natural_query)
    
    if result['success']:
        print(f"\nGenerated SQL:\n{result['sql_query']}\n")
        print(f"Retrieved {result['row_count']} rows")
        
        # Show data
        print("\nData:")
        display(result['results'])
        
        # Auto-visualize
        print("\nVisualization:")
        viz = auto_visualize(result['results'], natural_query)
        if viz is not None and hasattr(viz, 'show'):
            viz.show()
    else:
        print(f"Error: {result['error']}")

# 파이프라인 테스트
query_and_visualize("What is the total budget by department?")

## 7. 대시보드 스타일 다중 차트

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 하나의 그림에 여러 차트 생성
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "domain"}],
           [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=('Sales by Region', 'Employees by Department', 
                    'Department Budgets', 'Salary Distribution')
)

# 차트 1: 지역별 매출
df1 = db.execute_query_df("SELECT region, SUM(total_amount) as total FROM sales GROUP BY region")
fig.add_trace(go.Bar(x=df1['region'], y=df1['total'], name='Sales'), row=1, col=1)

# 차트 2: 부서별 직원
df2 = db.execute_query_df("SELECT d.department_name, COUNT(*) as count FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name")
fig.add_trace(go.Pie(labels=df2['department_name'], values=df2['count'], name='Employees'), row=1, col=2)

# 차트 3: 부서 예산
df3 = db.execute_query_df("SELECT department_name, budget FROM departments ORDER BY budget DESC")
fig.add_trace(go.Bar(x=df3['department_name'], y=df3['budget'], name='Budget'), row=2, col=1)

# 차트 4: 급여 분포
df4 = db.execute_query_df("SELECT salary FROM employees")
fig.add_trace(go.Histogram(x=df4['salary'], name='Salary'), row=2, col=2)

fig.update_layout(height=800, showlegend=False, title_text="Company Analytics Dashboard")
fig.show()

## 요약

이 노트북에서 배운 내용:
- ✓ 다양한 유형의 차트 생성 방법 (막대, 파이, 선, 산점도)
- ✓ 자동 시각화 사용 방법
- ✓ 쿼리에서 차트까지의 완전한 파이프라인 구축 방법
- ✓ 대시보드 스타일 시각화 생성 방법

다음 단계: `06_end_to_end.ipynb`로 이동하여 완전한 종단간 데모를 확인합니다.